<a href="https://www.kaggle.com/code/sweumavarsh/en-sa?scriptVersionId=232440762" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.9 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
!nvidia-smi

Tue Mar 11 12:42:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8             11W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import torch
print(torch.__version__)  # Check PyTorch version
print(torch.version.cuda)  # Check CUDA version
print(torch.backends.cudnn.version())  # Check cuDNN version
print(torch.cuda.is_available())  # Check if GPU is available
print(torch.cuda.device_count())  # Number of GPUs available
print(torch.cuda.get_device_name(0))  # GPU name (if available)

2.5.1+cu121
12.1
90100
True
2
Tesla T4


In [4]:
import pandas as pd
import torch
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, Trainer, TrainingArguments
from datasets import Dataset
import evaluate

In [5]:
# Load dataset
file_path = "/kaggle/input/trilingual-dataset/trilingual_dataset.xlsx"
xls = pd.ExcelFile(file_path)
train_df = pd.read_excel(xls, sheet_name="Train")
test_df = pd.read_excel(xls, sheet_name="Test")

In [6]:
# Preprocessing: Rename columns for consistency
train_df = train_df.rename(columns={"English": "en", "Hindi": "hi", "Sanskrit": "sa"})
test_df = test_df.rename(columns={"English": "en", "Hindi": "hi", "Sanskrit": "sa"})

In [7]:
# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [8]:
# Load Model & Tokenizer
model_name = "facebook/m2m100_418M"
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

In [9]:
# Tokenization function
def tokenize_function(examples):
    inputs = tokenizer(examples["en"], padding="max_length", truncation=True, max_length=128)
    targets = tokenizer(examples["sa"], padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs

In [10]:
# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2032 [00:00<?, ? examples/s]

In [11]:
training_args = TrainingArguments(
    output_dir="./m2m100_finetuned_sa",
    report_to="none",
    eval_strategy="epoch",  
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    push_to_hub=False,
)

In [12]:
# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
trainer.train(resume_from_checkpoint=False)  # Restart training instead of resuming

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


In [ ]:
pip install rouge_score

In [ ]:
model.save_pretrained("/kaggle/working/m2m100_finetuned_sa")
tokenizer.save_pretrained("/kaggle/working/m2m100_finetuned_sa")

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model_path = "/kaggle/working/m2m100_finetuned_sa"
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [ ]:
model.gradient_checkpointing_enable()

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
# Evaluation
import evaluate
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

In [ ]:
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    bleu_score = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    rouge_score = rouge.compute(predictions=decoded_preds, references=decoded_labels)

    # Print Scores
    print("\nEvaluation Metrics")
    print(f"BLEU Score: {bleu_score['bleu']:.4f}")
    print(f"ROUGE Scores: {rouge_score}")
    
    return {"bleu": bleu_score, "rouge": rouge_score}

In [ ]:
# Generate predictions and evaluate
results = trainer.evaluate()

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=1,  # Reduce batch size if needed
    report_to="none",  # Disable Weights & Biases logging
)

In [ ]:
# Extract predictions and labels from the dataset
eval_preds = trainer.predict(trainer.eval_dataset)

# Compute and print scores
compute_metrics(eval_preds)

# Print Scores
print("\nEvaluation Metrics")
print(f"BLEU Score: {bleu_score['bleu']:.4f}")
print(f"ROUGE Scores: {rouge_score}")

In [ ]:
# Save Model
model.save_pretrained('/kaggle/working/m2m100-en-hi-sa')
tokenizer.save_pretrained('/kaggle/working/m2m100-en-hi-sa')

!zip -r /kaggle/working/m2m100-en-hi-sa.zip /kaggle/working/m2m100-en-hi-sa
print('Model saved and zipped!')

from google.colab import drive
drive.mount('/content/drive')
!cp /kaggle/working/m2m100-en-hi-sa.zip /content/drive/MyDrive/

print("Fine-tuning complete! Model saved in drive.")

In [ ]:
import os
print(os.listdir("./m2m100_finetuned_sa")) 

In [ ]:
from transformers import AutoModel, AutoTokenizer

model_path = "./m2m100_finetuned_sa"  
model = AutoModel.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

print("Model and tokenizer loaded successfully!")

In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained("./m2m100_finetuned_sa/checkpoint-3750")
print("Resumed model from checkpoint-3750")

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
dummy_input = "Hello, I am Swetha"
inputs = tokenizer(dummy_input, return_tensors="pt")

output_tokens = model.generate(**inputs)
output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

print(output_text)